In [3]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

## Constants

In [4]:
# Change this to your location 
import variables
PATH = variables.stocks

In [9]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

risk_free_rate = 0.0125 # Approximate 10 year bond rate

## Get Stock File Names in a List

In [8]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

# On MacOS Only
# tickers.remove('.DS_Store')
# 3263 total stocks
tickers.sort()
len(tickers)

3261

## Function that Returns a Dataframe from a CSV


In [14]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError as ex:
        print(ex)
    else:
        return df

## Merge Multiple Stocks in One Dataframe by Column Name


In [15]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_stock_df_from_csv(x)
        
        # NEW Check if your dataframe has duplicate indexes
        # if not df.index.is_unique:
        #     # Delete duplicates 
        #     df = df.loc[~df.index.duplicated(), :]
        
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

## Markowitz Portfolio Optimization

Harry Markowitz proved that you could make what is called an efficient portfolio. That is a portfolio that optimizes return while also minimizing risk. We don't benefit from analyzing individual securities at the same rate as if we instead considered a portfolio of stocks.

We do this by creating portfolios with stocks that are not correlated. We want to calculate expected returns by analyzing the returns of each stock multiplied by its weight. 

$w_1r_1 + w_2r_2 = r_p$

The standard deviation of the portfolio is found this way. Sum multiple calculations starting by finding the product of the first securities weight squared times its standard deviation squared. The middle is 2 times the correlation coefficient between the stocks. And, finally add those to the weight squared times the standard deviation squared for the second security.

$(w_1\sigma_1 + w_2\sigma_2)^2 = w_1^2\sigma_1^2 + 2w_1\sigma_1w_2\sigma_2\rho_1 + w_2^2\sigma_2^2$

## Plotting an Efficient Frontier


## Stock Portfolio

In [16]:
# 1ST LIST port_list = ['CALX', 'NOVT', 'BLDR', 'DXCM', 'RGEN', 'PFE', 'LLY',
# 'AMD', 'NVDA', 'NFLX', 'DAR', 'COST', 'BJ', 'WMT', 'BOOT', 'WING',
# 'LULU', 'NEE', 'AWK', 'MSCI', 'KKR', 'LPX', 'SHW', 'NXRT', 'NSA',
# 'CBRE', 'AES']

# 2ND LIST port_list = ['CALX', 'NOVT', 'RGEN', 'LLY',
# 'AMD', 'NFLX', 'COST', 'BJ', 'WING',
# 'LULU', 'NEE', 'MSCI', 'KKR', 'SHW', 'NSA',
# 'CBRE']
port_list = ['CALX', 'NOVT', 'RGEN', 'LLY',
'AMD', 'NFLX', 'COST', 'BJ', 'WING',
'MSCI', 'CBRE']
num_stocks = len(port_list)
num_stocks

11

## Merge All Data by Closing Price

In [17]:
mult_df = merge_df_by_column_name('Close',  S_DATE, 
                                  E_DATE, *port_list)
mult_df

,CALX,NOVT,RGEN,LLY,AMD,NFLX,COST,BJ,WING,MSCI,CBRE
Date,,,,,,,,,,,
2017-02-08,6.900000,23.500000,29.090000,70.125069,13.560000,144.740005,150.527267,NaN,24.226603,87.968056,30.780001
2017-02-09,7.000000,24.000000,30.270000,69.756973,13.420000,144.139999,152.797440,NaN,24.486109,85.802345,31.570000
2017-02-10,7.400000,24.400000,30.070000,69.837784,13.580000,144.820007,153.726959,NaN,24.519596,86.647743,34.000000
2017-02-13,7.300000,24.600000,30.110001,70.289688,13.490000,143.199997,153.691208,NaN,24.310310,88.139053,34.490002
2017-02-14,7.650000,24.549999,30.350000,71.229660,13.260000,140.820007,155.183792,NaN,24.084288,89.240898,35.130001
...,...,...,...,...,...,...,...,...,...,...,...
2022-02-02,50.380001,138.919998,204.130005,250.830002,122.760002,429.480011,521.230042,60.930000,153.410004,549.830017,103.230003
2022-02-03,49.340000,134.000000,198.360001,244.809998,120.080002,405.600006,521.770020,60.910000,150.360001,540.309998,102.120003
2022-02-04,50.150002,137.470001,200.949997,242.270004,123.599998,410.170013,519.770020,59.849998,149.339996,554.710022,101.230003


## Plot Price of Investments over Total Dataset


In [19]:
fig = px.line(mult_df, x=mult_df.index, y=mult_df.columns)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Price")
fig.update_layout(height=1200, width=1800, 
                  showlegend=True)
fig.show()

## Cumulative Return of all Stocks

In [20]:
mult_cum_df = merge_df_by_column_name('cum_return', S_DATE, E_DATE, *port_list)
mult_cum_df

,CALX,NOVT,RGEN,LLY,AMD,NFLX,COST,BJ,WING,MSCI,CBRE
Date,,,,,,,,,,,
2017-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-09,1.014493,1.021277,1.040564,0.994751,0.989675,0.995855,1.015081,NaN,1.010712,0.975381,1.025666
2017-02-10,1.072464,1.038298,1.033689,0.995903,1.001475,1.000553,1.021257,NaN,1.012094,0.984991,1.104613
2017-02-13,1.057971,1.046809,1.035064,1.002348,0.994838,0.989360,1.021019,NaN,1.003455,1.001944,1.120533
2017-02-14,1.108696,1.044681,1.043314,1.015752,0.977876,0.972917,1.030935,NaN,0.994126,1.014469,1.141326
...,...,...,...,...,...,...,...,...,...,...,...
2022-02-02,7.301449,5.911489,7.017188,3.576895,9.053097,2.967252,3.462695,2.769545,6.332295,6.250337,3.353801
2022-02-03,7.150725,5.702128,6.818838,3.491048,8.855457,2.802266,3.466282,2.768636,6.206401,6.142116,3.317739
2022-02-04,7.268116,5.849787,6.907872,3.454827,9.115044,2.833840,3.452996,2.720454,6.164298,6.305812,3.288824


In [25]:
fig = px.line(mult_cum_df, x=mult_cum_df.index, y=mult_cum_df.columns)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Price")
fig.update_layout(height=1000, width=1800, 
                  showlegend=True)
fig.show()

## Mean Returns

In [26]:
returns = np.log(mult_df / mult_df.shift(1))
mean_ret = returns.mean() * 252
mean_ret

CALX    0.405799
NOVT    0.358622
RGEN    0.380407
LLY     0.245146
AMD     0.442498
NFLX    0.204248
COST    0.246863
BJ      0.281779
WING    0.365446
MSCI    0.361443
CBRE    0.242308
dtype: float64

## Correlation